In [2]:
import sys
sys.path.append("../..")

from lunar_crater_age_logic.preprocess import load_data
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np


2025-12-10 16:32:31.831559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/santanu/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/santanu/.pyenv/versions/3.10.6/lib/python3.10/runpy

AttributeError: _ARRAY_API not found

In [5]:
# Configuration
DATA_DIR = Path("/home/santanu/code/VMontejo/lunar-crater-age-classifier/raw_data/train")
IMG_HEIGHT = 227  # Changed to match preprocess.py
IMG_WIDTH = 227   # Changed to match preprocess.py
BATCH_SIZE = 32
EPOCHS = 50
NUM_CLASSES = 3

In [6]:
# Load data using your preprocess.py function
print("Loading data from preprocess.py...")

# Load full dataset with weighted sampling (handles imbalance)
train_loader = load_data(
    data_dir=DATA_DIR,
    balanced=False,  # Use full dataset
    batch_size=BATCH_SIZE,
    use_weighted_sampling=True,  # Handles class imbalance
    seed=42
)

# For validation, create a separate loader without shuffling
val_loader = load_data(
    data_dir=DATA_DIR,
    balanced=False,
    batch_size=BATCH_SIZE,
    use_weighted_sampling=False,
    seed=123  # Different seed for validation
)

class_names = ["ejecta", "oldcrater", "none"]
print(f"Class names: {class_names}")


Loading data from preprocess.py...
Creating FULL dataset (all available data)
ejecta: 358 images
oldcrater: 594 images
none: 2656 images
Total images 3608
Applying weighted sampling strategy
Original class distribution:
ejecta: 358 samples
oldcrater: 594 samples
none: 2656 samples
After weighted resampling
ejecta: 2656 samples (weight: 7.42)
oldcrater: 2656 samples (weight: 4.47)
none: 2656 samples (weight: 1.00)
Creating FULL dataset (all available data)
ejecta: 358 images
oldcrater: 594 images
none: 2656 images
Total images 3608
Using imbalanced data without weighting
Class names: ['ejecta', 'oldcrater', 'none']


In [13]:
def build_lroc_model(input_shape, num_classes):
    model = models.Sequential(name="LROC_Custom_CNN_RGB")

    model.add(layers.InputLayer(shape=input_shape))

    # --- Block 1 ---
    model.add(layers.Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 2 ---
    model.add(layers.Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 3 ---
    model.add(layers.Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 4 ---
    model.add(layers.Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 5 (NEW): deeper crater textures ---
    model.add(layers.Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))


    # --- Classification ---
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

    # Output layer must be softmax
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [14]:

# Build model with correct input shape
model = build_lroc_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
model.summary()

2025-12-10 16:02:22.123967: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "LROC_Custom_CNN_RGB"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 227, 227, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 227, 227, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 227, 227, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 113, 113, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 113, 113, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 113, 113, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 14, 14, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,32

 Total params: 1,704,643 (6.50 MB)

 Trainable params: 1,702,659 (6.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [15]:
def sparse_softmax_focal_loss(gamma=2.0, alpha=[1.8315, 0.6731, 1.4221]):
    """
    Focal Loss for multi-class classification with sparse labels.
    """

    alpha = tf.constant(alpha, dtype=tf.float32)  # <--- move this outside the inner function

    def loss_fn(y_true, y_pred):

        # Cast labels
        y_true = tf.cast(y_true, tf.int32)

        # One-hot encode
        num_classes = y_pred.shape[-1]
        y_true_onehot = tf.one_hot(y_true, depth=num_classes)

        # Numerical stability
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)

        # Cross-entropy
        ce = -tf.reduce_sum(y_true_onehot * tf.math.log(y_pred), axis=-1)

        # p_t
        p_t = tf.reduce_sum(y_true_onehot * y_pred, axis=-1)

        # Focal modulation
        modulating_factor = tf.pow((1 - p_t), gamma)

        # Alpha weighting (no name conflict now)
        alpha_t = tf.reduce_sum(y_true_onehot * alpha, axis=-1)

        return alpha_t * modulating_factor * ce

    return loss_fn

In [16]:
# 1. Compile

optimizer = optimizers.Adam(learning_rate=0.0001,
                            global_clipnorm=1.0,
)
focal_loss = sparse_softmax_focal_loss(
    gamma=2.0,
    alpha=[1.8315, 0.6731, 1.4221]
)


model.compile(
    optimizer=optimizer,
    loss=focal_loss,
    metrics=['accuracy']
)

# --- FIX 2: ROBUST CALLBACKS ---
callbacks = [
    # Stop training if validation loss doesn't improve for 5 epochs
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),

    # Save the best model automatically in the native Keras format
    ModelCheckpoint('best_lroc_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),

    # Slow down learning rate if the model gets stuck
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
]

print("✅ Model compiled. Callbacks configured to save as .keras")

✅ Model compiled. Callbacks configured to save as .keras


In [ ]:
# Train model
print("Starting training...")
history = model.fit(
    train_loader,
    epochs=EPOCHS,
    validation_data=val_loader,
    callbacks=callbacks,
    verbose=1
)

print("✅ Training Complete.")